In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Monitoring PyTorch experiments with AI Platform TensorBoard

This code sample demonstrates how to configure AI Platform TensorBoard (Experimental) to monitor PyTorch training jobs.

AI Platform Tensorboard is an enterprise ready, managed version of TensorBoard, a Google Open Source project for Machine Learning experiment visualization, that is tightly integrated with the Google Cloud AI Platform.

TensorBoard provides the visualization and tooling needed for machine learning experimentation:
* Tracking and visualizing metrics such as loss and accuracy
* Visualizing the model graph (ops and layers)
* Viewing histograms of weights, biases, or other tensors as they change over time
* Projecting embeddings to a lower dimensional space
* Displaying images, text, and audio data
* Profiling TensorFlow programs
* And much more

Note that currently, AI Platform TensorBoard requires AI Platform Training and only supports the Scalars dashboard. As support for other features of TensorBoard are added this notebook will be updated.


## Pre-requisites

This notebook was designed to run on [AI Platform Notebooks](https://cloud.google.com/ai-platform/notebooks/docs) using the standard PyTorch 1.6+ image. Your notebook instance should be in the same project as the AI Platform TensorBoard and Training services.

While AI Platform TensorBoard is in the Experimental stage your project must be allow-listed before using the service. Use the [signup form](https://docs.google.com/forms/d/e/1FAIpQLSfbvZ5xrFStX54qEUlJ6A0tWZ-O20i_t-Hifm0JvbX8do5IcQ/viewform) to request the acccess.

After your project has been allow-listed make sure to [enable Cloud AI Platform API](https://console.cloud.google.com/apis/library/aiplatform.googleapis.com?q=cloud%20ai%20platform%20api&id=6189b0c0-23b1-46a4-a32f-70639e83fe9b).

## Scenario

You will use AI Platform TensorBoard to monitor PyTorch AI Platform Training jobs. The training scenario is transfer learning for an image classification problem, inspired by the Kaggle's classic Dogs vs. Cats competition. 

You will run and monitor two types of AI Platform Training jobs: a custom training job and a hyperparameter tuning job. Both types of jobs will utilize the same custom docker image that encapsulates the training application and the PyTorch runtime.



## Setting up the environment

In [1]:
import base64
import os
import json
import time
import numpy as np

import google.auth

from google.auth.credentials import Credentials
from google.auth.transport.requests import AuthorizedSession

from typing import List, Optional, Text, Tuple

### Create AI Platform Training service account



To integrate with AI Platform TensorBoard, AI Platform Training jobs have to run in the context a service account that has permisions to write logs to GCS and access the AI Platform TensorBoard service. 

If you don't have one already set up, create and configure a new service account using the following instructions. Note that by default, your AI Platform Notebook instance is running under the **Compute Engine default service account** that does not have the required permissions (specifically `iam.serviceAccounts.setIamPolicy`) to configure the service account. If this is the case, use a different environment  (for example **Cloud Shell**) and the account with the required permissions to run the below commands.


1. Create a service account

```
PROJECT_ID=[YOUR_PROJECT_ID]
USER_SA_NAME=[YOUR_SA_ACCOUNT_NAME]

gcloud --project=$PROJECT_ID iam service-accounts create $USER_SA_NAME

```

2. Retrieve the internal service account used by AI Platform 

```
GOOGLE_SA=$(gcloud projects get-iam-policy $PROJECT_ID \
    --flatten="bindings[].members" --format="table(bindings.members)" \
    --filter="bindings.role:roles/aiplatform.customCodeServiceAgent" | \
    grep "serviceAccount:" | head -n1)
```

3. Give the AI Platform service account permissions to impersonate your service account

```
SA_EMAIL="${USER_SA_NAME}@${PROJECT_ID}.iam.gserviceaccount.com"

gcloud --project=$PROJECT_ID iam service-accounts add-iam-policy-binding \
    --role roles/iam.serviceAccountAdmin \
    --member $GOOGLE_SA $SA_EMAIL

```

4. Give your service account access to GCS and AI Platform Tensorboard service.

```
gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member="serviceAccount:${SA_EMAIL}" \
    --role="roles/storage.admin"

gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member="serviceAccount:${SA_EMAIL}" \
    --role="roles/aiplatform.user"

```


Set the `SA_EMAIL` constant to the email account of your service account. If you followed the above steps to create the account you can display it by:

```
echo $SA_EMAIL
```

In [26]:
SA_EMAIL = 'aip-training@jk-mlops-dev.iam.gserviceaccount.com'

### Set AI Platform (Unified) constants

`PROJECT_ID` - The GCP Project ID. Both your AI Platform Notebook instance and AI Platform Training jobs should be in the same project. Make sure to modify the placeholder with your Project ID.

`CAIP_REGION` - A GCP compute region to use for the cloud services used in this notebook. Make sure to choose a region where [Cloud AI Platform services](https://cloud.google.com/ml-engine/docs/tensorflow/regions) are available. The default region is `us-central1`. 


In [27]:
PROJECT_ID = 'jk-mlops-dev'
CAIP_REGION = 'us-central1'

The current version of AI Platform (Unified) Python SDK does not support the AI Platform TensorBoard service nor the AI Platform Training service intergration with AI Platform TensorBoard. To mitigate, the notebook calls the APIs directly using the REST interface. You don't need to change the below constants that define the API's endpoint and root resource paths. 

`CAIP_ENDPOINT` - The AI Platform (Unified) API service endpoint.

`CAIP_PARENT_ALPHA` - The AI Platform (Unified) Alpha (Experimental) API root resource path. AI Platform (Unified) TensorBoard is in the Experimental stage.

`CAIP_PARENT_BETA` - The AI Platform (Unified) Beta (Preview) API root resource path. AI Platform (Unified) Training is in the Preview stage.


In [28]:
CAIP_ENDPOINT = f'{CAIP_REGION}-aiplatform.googleapis.com'
CAIP_PARENT_ALPHA = f'https://{CAIP_ENDPOINT}/v1alpha1/projects/{PROJECT_ID}/locations/{CAIP_REGION}'
CAIP_PARENT_BETA = f'https://{CAIP_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{CAIP_REGION}'

### Create a GCS bucket to store TensorBoard logs

The training script writes TensorBoard logs to a GCS bucket from which the AI Platform Training service  ingests them to the TensorBoard service. The GCS bucket should be a regional bucket  in the same region where the training job is executed.


In [29]:
GCS_BUCKET_NAME = f'{PROJECT_ID}-tensorboard-logs'

!gsutil mb -l {CAIP_REGION} gs://{GCS_BUCKET_NAME}

Creating gs://jk-mlops-dev-tensorboard-logs/...
ServiceException: 409 Bucket jk-mlops-dev-tensorboard-logs already exists.


### Create a TensorBoard instance

You will now create an AI Platform TensorBoard instance. As noted before you will the REST interface to invoke the APIs.

#### Create an authorized session 

In [30]:
credentials, _ = google.auth.default()
authed_session = AuthorizedSession(credentials)

#### Create a TensorBoard resorce

The following REST call creates a TensorBoard resource and sets its display name. Note that multiple resources can share the same display name, so each execution of the following cell creates a new TensorBoard instance.

In [40]:
tensorboard_display_name = 'pytorch_tensorboard'
api_url = f'{CAIP_PARENT_ALPHA}/tensorboards'

request_body = {
    'display_name': tensorboard_display_name
}

response = authed_session.post(api_url, data=json.dumps(request_body))
response.json()

{'name': 'projects/895222332033/locations/us-central1/tensorboards/7266276523785584640/operations/7568055082214752256',
 'metadata': {'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.CreateTensorboardOperationMetadata',
  'genericMetadata': {'createTime': '2020-12-02T23:18:20.643780Z',
   'updateTime': '2020-12-02T23:18:20.643780Z'}}}

#### List TensorBoard instances 

To verify that the instance was succesfully created list all instances with the set display name.

In [41]:
api_url = f'{CAIP_PARENT_ALPHA}/tensorboards?filter=display_name={tensorboard_display_name}'

response = authed_session.get(api_url)
response.json()

{'tensorboards': [{'name': 'projects/895222332033/locations/us-central1/tensorboards/7266276523785584640',
   'displayName': 'pytorch_tensorboard',
   'createTime': '2020-12-02T23:18:20.643780Z',
   'updateTime': '2020-12-02T23:18:20.736959Z',
   'etag': 'AMEw9yPK-DNWAfpIRfcdqxlcQtrHYxvsYyiIDg2JUARjIKR2M5-DefMrXtUdr34lmsMl'}]}

#### Retrieve a full name of your instance

You can retrieve the fullname of the created instance from the JSON response. 

In [42]:
tensorboard_id = response.json()['tensorboards'][0]['name']
tensorboard_id

'projects/895222332033/locations/us-central1/tensorboards/7266276523785584640'

Your environment is ready. You will now prepare a custom PyTorch training container and submit and monitor AI Platform Training jobs.

## Preparing a training container

### Create a training script

In [ ]:
%%writefile train_eval.py

# Copyright 2020 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and

import argparse
import hypertune
import numpy as np
import time
import os
import copy
import matplotlib.pyplot as plt
import zipfile

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision import datasets, models, transforms


DEFAULT_ROOT = '/tmp'

def get_catsanddogs(root):
    """
    Creates training and validation Datasets based on images
    of cats and dogs from 
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip.
    """
    
    # Download and extract the images
    source_url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
    local_filename = source_url.split('/')[-1]
    datasets.utils.download_url(source_url, root, )
    path_to_zip = os.path.join(root, local_filename)
    with zipfile.ZipFile(path_to_zip, 'r') as zip_ref:
        zip_ref.extractall(root)
    
    
    # Create datasets
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    
    val_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    
    train_dataset = datasets.ImageFolder(
        root=os.path.join(path_to_zip[:-4], 'train'),
        transform=train_transforms)
    
    val_dataset = datasets.ImageFolder(
        root=os.path.join(path_to_zip[:-4], 'validation'),
        transform=val_transforms
    )
    
    return train_dataset, val_dataset
    


def get_model(num_layers, dropout_ratio, num_classes):
    """
    Creates a convolution net using ResNet50 trunk and
    a custom head.
    """

    # Create the ResNet50 trunk
    model = models.resnet18(pretrained=True)

    # Get the number of input features to the default head
    num_features = model.fc.in_features

    # Freeze trunk weights
    for param in model.parameters():
        param.requires_grad = False

    # Define the new head
    head = nn.Sequential(nn.Linear(num_features, num_layers),
                         nn.ReLU(),
                         nn.Dropout(dropout_ratio),
                         nn.Linear(num_layers, num_classes))

    # Replace the head
    model.fc = head

    return model


def train_eval(device, model, train_dataloader, valid_dataloader,
               criterion, optimizer, scheduler, num_epochs, writer=None):
    """
    Trains and evaluates a model.
    """
    since = time.time()

    model = model.to(device)

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    hpt = hypertune.HyperTune()

    for epoch in range(1, num_epochs+1):

        # Training phase
        model.train()
        num_train_examples = 0
        train_loss = 0.0

        for inputs, labels in train_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            num_train_examples += inputs.size(0)
            train_loss += loss.item() * inputs.size(0)
        scheduler.step()

        # Validation phase
        model.eval()
        num_val_examples = 0
        val_loss = 0
        val_corrects = 0

        for inputs, labels in valid_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)

            num_val_examples += inputs.size(0)
            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(torch.eq(torch.max(outputs, 1)
                                               [1], labels))

        # Log epoch metrics
        train_loss = train_loss / num_train_examples
        val_loss = val_loss / num_val_examples
        val_acc = val_corrects.double() / num_val_examples

        print('Epoch: {}/{}, Training loss: {:.3f}, Validation loss: {:.3f}, Validation accuracy: {:.3f}'.format(
              epoch, num_epochs, train_loss, val_loss, val_acc))

        # Write to Tensorboard
        if writer:
            writer.add_scalars(
                'Loss', {'training': train_loss, 'validation': val_loss}, epoch)
            writer.add_scalar('Validation accuracy', val_acc, epoch)
            writer.flush()
            
        # Report to HyperTune
        hpt.report_hyperparameter_tuning_metric(
            hyperparameter_metric_tag='accuracy',
            metric_value=val_acc,
            global_step=epoch
        )

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_acc


def get_args():
    """
    Returns parsed command line arguments.
    """

    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--num-epochs',
        type=int,
        default=20,
        help='number of times to go through the data, default=20')
    parser.add_argument(
        '--batch-size',
        default=32,
        type=int,
        help='number of records to read during each training step, default=32')
    parser.add_argument(
        '--num-layers',
        default=64,
        type=int,
        help='number of hidden layers in the classification head , default=64')
    parser.add_argument(
        '--dropout-ratio',
        default=0.5,
        type=float,
        help='dropout ration in the classification head , default=128')
    parser.add_argument(
        '--step-size',
        default=7,
        type=int,
        help='step size of LR scheduler')
    parser.add_argument(
        '--log-dir',
        type=str,
        default='/tmp',
        help='directory for TensorBoard logs')
    parser.add_argument(
        '--verbosity',
        choices=['DEBUG', 'ERROR', 'FATAL', 'INFO', 'WARN'],
        default='INFO')

    args, _ = parser.parse_known_args()
    return args


if __name__ == "__main__":
    
    # Parse command line arguments
    args = get_args()
    
    # Create train and validation dataloaders
    train_dataset, val_dataset = get_catsanddogs(DEFAULT_ROOT)
    train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=True)
    class_names = train_dataset.classes
    
    # Use GPU if available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print('-' * 10)
    print(f'Training on device: {device}')

    # Configure training
    model = get_model(args.num_layers, args.dropout_ratio, len(class_names))
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer, step_size=args.step_size, gamma=0.1)

    # Set location for the TensorBoard logs
    if 'AIP_TENSORBOARD_LOG_DIR' in os.environ:
        log_dir = os.environ['AIP_TENSORBOARD_LOG_DIR']
    else:
        log_dir = args.log_dir

    with SummaryWriter(log_dir) as writer:
        trained_model, accuracy = train_eval(device, model, train_dataloader, val_dataloader,
                                             criterion, optimizer, scheduler, args.num_epochs, writer)



### Create Dockerfile

In [ ]:
%%writefile Dockerfile

FROM gcr.io/deeplearning-platform-release/pytorch-gpu.1-6
    
RUN pip install -U tensorflow cloudml-hypertune

ADD train_eval.py .

ENTRYPOINT ["python3", "train_eval.py"]


### Build the image

In [ ]:
image_name = 'image_classifier'
image_tag = 'latest'
image_uri = f'gcr.io/{project_id}/{image_name}:{image_tag}'

#!gcloud builds submit --tag {image_uri} .

## Submitting training jobs

### Submit a custom training job

In [ ]:
job_name = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
base_output_dir = f'gs://{gcs_bucket_name}/{job_name}'
tb_name = 'projects/895222332033/locations/us-central1/tensorboards/1989183660414205952'
sa_email = 'aip-training@jk-mlops-dev.iam.gserviceaccount.com'

num_epochs = 10

In [ ]:
api_url = f'{beta_api_prefix}/customJobs'

request_body = {
    'display_name': job_name,
    'job_spec': {
        'worker_pool_specs': [
            {
                'replica_count': 1,
                'machine_spec': {
                    'machine_type': 'n1-standard-8',
                    'accelerator_type': 'NVIDIA_TESLA_T4',
                    'accelerator_count': 1
                },
                'container_spec': {
                    'image_uri': image_uri,
                    'args': [
                        f'--num-epochs={num_epochs}'
                    ]
                }
            }
        ],
        'base_output_directory': {
            'output_uri_prefix': base_output_dir,
        },
        'service_account': sa_email,
        'tensorboard': tb_name
    }
}


response = authed_session.post(api_url, data=json.dumps(request_body))
response.json()

### Submit a hyperparameter tuning job

In [ ]:
api_url = f'{beta_api_prefix}/hyperparameterTuningJobs'

request_body = {
    'display_name': job_name,
    'study_spec' : {
        'metrics': [
            {
                'metric_id': 'accuracy',
                'goal': 'MAXIMIZE'
            }
        ],
        'parameters': [
            {
                'parameter_id': 'batch-size',
                'discrete_value_spec': {'values': [32, 64, 128]},
                'scale_type': 'UNIT_LINEAR_SCALE'
            },
            {
                'parameter_id': 'num-layers',
                'discrete_value_spec': {'values': [64, 128]},
                'scale_type': 'UNIT_LINEAR_SCALE'
            }
        ],
    'algorithm':'GRID_SEARCH'
    },
    'maxTrialCount': 6,
    'parallelTrialCount': 3,
    'trial_job_spec': {
        'worker_pool_specs': [
            {
                'replica_count': 1,
                'machine_spec': {
                    'machine_type': 'n1-standard-8',
                    'accelerator_type': 'NVIDIA_TESLA_T4',
                    'accelerator_count': 1
                },
                'container_spec': {
                    'image_uri': image_uri,
                    'args': [
                        f'--num-epochs={num_epochs}'
                    ]
                }
            }
        ],
        'base_output_directory': {
            'output_uri_prefix': base_output_dir,
        },
        'service_account': sa_email,
        'tensorboard': tb_name
    }
}


response = authed_session.post(api_url, data=json.dumps(request_body))
response.json()

#request_body

## Cleaning up

### List all tensorboards in the project

In [39]:
api_url = f'{CAIP_PARENT_ALPHA}/tensorboards'

response = authed_session.get(api_url)
response.json()

{}

### Delete a TensorBoard resource

In [38]:
tensorboard_id = '6694319371109531648'

api_url = f'{CAIP_PARENT_ALPHA}/tensorboards/{tensorboard_id}'

response = authed_session.delete(api_url)
response.json()

{'name': 'projects/895222332033/locations/us-central1/operations/3952790481343086592',
 'metadata': {'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.DeleteOperationMetadata',
  'genericMetadata': {'createTime': '2020-12-02T23:18:08.357755Z',
   'updateTime': '2020-12-02T23:18:08.357755Z'}},
 'done': True,
 'response': {'@type': 'type.googleapis.com/google.protobuf.Empty'}}

# Parking lot

## Visualize a few images

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(trainiter)

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

## Define a model

In [ ]:
def get_model(num_layers, dropout_ratio, num_classes):
    """
    Creates a convolution net using ResNet50 trunk and
    a custom head.
    """
    
    # Get the number of input features to the default head
    model = models.resnet50(pretrained=True)
    num_features = model.fc.in_features
    
    # Freeze trunk weights
    for param in model.parameters():
        param.requires_grad = False
    
    # Define a new head
    head = nn.Sequential(nn.Linear(num_features, num_layers),
                         nn.ReLU(),
                         nn.Dropout(dropout_ratio),
                         nn.Linear(num_layers, num_classes))
    
    # Replace the head
    model.fc = head
    
    return model
    

In [ ]:
num_layers = 64
dropout_ratio = 0.5
num_classes = 2

model = get_model(num_layers, dropout_ratio, num_classes)


## Inspect the model

In [ ]:
model

In [ ]:
model.fc

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

## Define a training loop

In [ ]:
from torch.utils.tensorboard import SummaryWriter

def train_model(device, dataloaders, dataset_sizes, model, criterion, 
                optimizer, scheduler, num_epochs=25, log_dir='/tmp'):
    
    since = time.time()
    writer = SummaryWriter(log_dir)
    
    
    

    model = model.to(device)
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            # Write loss and accuracy to TensorBoard
            writer.add_scalar(f'Loss/{phase}', epoch_loss, epoch)
            writer.add_scalar(f'Acc/{phase}', epoch_acc, epoch)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    writer.close()
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

## Start training

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


data_dir = '/tmp/data/hymenoptera_data'
batch_size = 32
log_dir = 'gs://jk-tensorboards/experiments/102'
num_epochs = 50


In [ ]:
dataloaders, datasizes, class_names = get_dataloaders(data_dir, batch_size)

model = train_model(device, dataloaders, datasizes, model, criterion, 
                    optimizer_ft, exp_lr_scheduler, num_epochs, log_dir)

## TensorBoard setup

In [ ]:
from torch.utils.tensorboard import SummaryWriter


log_dir = 'gs://jk-tensorboards/experiments/1'

writer = SummaryWriter(log_dir)

In [ ]:
# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# create grid of images
img_grid = torchvision.utils.make_grid(images)

# show images
matplotlib_imshow(img_grid, one_channel=True)

# write to tensorboard
writer.add_image('four_fashion_mnist_images', img_grid)


In [ ]:
writer.add_graph(net, images)
writer.close()

In [ ]:
def select_n_random(data, labels, n=100):
    '''
    Selects n random datapoints and their corresponding labels from a dataset
    '''
    assert len(data) == len(labels)

    perm = torch.randperm(len(data))
    return data[perm][:n], labels[perm][:n]

# select random images and their target indices
images, labels = select_n_random(trainset.data, trainset.targets)

# get the class labels for each image
class_labels = [classes[lab] for lab in labels]

# log embeddings
features = images.view(-1, 28 * 28)
writer.add_embedding(features,
                    metadata=class_labels,
                    label_img=images.unsqueeze(1))
writer.close()

In [ ]:
def images_to_probs(net, images):
    '''
    Generates predictions and corresponding probabilities from a trained
    network and a list of images
    '''
    output = net(images)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.numpy())
    return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


def plot_classes_preds(net, images, labels):
    '''
    Generates matplotlib Figure using a trained network, along with images
    and labels from a batch, that shows the network's top prediction along
    with its probability, alongside the actual label, coloring this
    information based on whether the prediction was correct or not.
    Uses the "images_to_probs" function.
    '''
    preds, probs = images_to_probs(net, images)
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(12, 48))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        matplotlib_imshow(images[idx], one_channel=True)
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            classes[preds[idx]],
            probs[idx] * 100.0,
            classes[labels[idx]]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig

In [ ]:
running_loss = 0.0
for epoch in range(1):  # loop over the dataset multiple times

    for i, data in enumerate(trainloader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999:    # every 1000 mini-batches...

            # ...log the running loss
            writer.add_scalar('training loss',
                            running_loss / 1000,
                            epoch * len(trainloader) + i)

            # ...log a Matplotlib Figure showing the model's predictions on a
            # random mini-batch
            writer.add_figure('predictions vs. actuals',
                            plot_classes_preds(net, inputs, labels),
                            global_step=epoch * len(trainloader) + i)
            running_loss = 0.0
print('Finished Training')

In [ ]:
# 1. gets the probability predictions in a test_size x num_classes Tensor
# 2. gets the preds in a test_size Tensor
# takes ~10 seconds to run
class_probs = []
class_preds = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        output = net(images)
        class_probs_batch = [F.softmax(el, dim=0) for el in output]
        _, class_preds_batch = torch.max(output, 1)

        class_probs.append(class_probs_batch)
        class_preds.append(class_preds_batch)

test_probs = torch.cat([torch.stack(batch) for batch in class_probs])
test_preds = torch.cat(class_preds)

# helper function
def add_pr_curve_tensorboard(class_index, test_probs, test_preds, global_step=0):
    '''
    Takes in a "class_index" from 0 to 9 and plots the corresponding
    precision-recall curve
    '''
    tensorboard_preds = test_preds == class_index
    tensorboard_probs = test_probs[:, class_index]

    writer.add_pr_curve(classes[class_index],
                        tensorboard_preds,
                        tensorboard_probs,
                        global_step=global_step)
    writer.close()

# plot all the pr curves
for i in range(len(classes)):
    add_pr_curve_tensorboard(i, test_probs, test_preds)